In [ ]:
# from collections import Counter
# import numpy as np

# # Step 1: Preprocessing for cumulative counts and Occurrence matrix
# def preprocess(L):
#     # Compute cumulative counts (C)
#     counts = Counter(L)
#     sorted_chars = sorted(counts.keys())
#     C = {}
#     running_total = 0
#     for char in sorted_chars:
#         C[char] = running_total
#         running_total += counts[char]
    
#     # Compute Occurrence matrix (Occ)
#     Occ = {char: np.zeros(len(L) + 1, dtype=int) for char in sorted_chars}
#     for i, char in enumerate(L):
#         for c in sorted_chars:
#             Occ[c][i + 1] = Occ[c][i] + (1 if char == c else 0)
    
#     return C, Occ

# # Preprocess L to get C and Occ
# C, Occ = preprocess(L)

# # Step 2: Backward Search using C and Occ
# def backward_search_wheeler(query, C, Occ, L):
#     range_start = 0
#     range_end = len(L) - 1  # 0-based indexing
    
#     for char in reversed(query):
#         if char not in C:
#             return None  # Character not found in graph
        
#         # Update the range using C and Occ
#         range_start = C[char] + Occ[char][range_start]
#         range_end = C[char] + Occ[char][range_end + 1] - 1
        
#         if range_start > range_end:
#             return None  # Query not found
    
#     return range_start, range_end

# # Test the algorithm with a query string
# query_string = "your"
# result_wheeler = backward_search_wheeler(query_string, C, Occ, L)
# result_wheeler


NameError: name 'L' is not defined

In [39]:
# class WheelerGraphIndex:
#     def __init__(self, L, I, O, C, wt):
#         """
#         L: array of edge labels (or could be implicit in wavelet tree wt)
#         I, O: bitvectors representing incoming/outgoing edges to nodes
#         C: dictionary mapping character -> starting index in 'first column'
#         wt: wavelet tree or structure supporting rank/select on L
#         """
#         self.L = L
#         self.I = I
#         self.O = O
#         self.C = C
#         self.wt = wt
#         self.num_edges = len(L)

#     def rank_c(self, c, pos):
#         # rank_c function using wavelet tree:
#         # returns how many times 'c' appears in L up to index pos
#         return self.wt.rank(c, pos)

#     def backward_search(self, P):
#         """
#         Perform a backward search for pattern P on the Wheeler Graph.
#         Returns the range [low, high] of edges that match P. If no match,
#         returns an empty range.
#         """
#         low, high = 0, self.num_edges - 1
#         for i in range(len(P) - 1, -1, -1):
#             c = P[i]
#             # Get the count of c in [0..low-1] and [0..high]
#             if low > 0:
#                 lRank = self.rank_c(c, low - 1)
#             else:
#                 lRank = 0
#             rRank = self.rank_c(c, high)

#             if rRank == lRank:
#                 # no occurrences of c in [low..high]
#                 return (-1, -1)  # no match

#             # Update the range using the C array
#             low = self.C[c] + lRank
#             high = self.C[c] + rRank - 1

#         return (low, high)

#     def paths_for_range(self, low, high):
#         """
#         Given a final range of edges matching P, recover the paths.
#         This step is more dependent on your Wheeler Graph representation.
#         Typically, you'd use I and O bitvectors to map edge indices back
#         to nodes and reconstruct the paths. For now, we'll just return
#         the edge indices.
#         """
#         # This is just a stub. Actual logic depends on how I and O map edges to nodes.
#         return list(range(low, high+1))

# # Example usage (assuming you already built the structures):
# # Suppose P is the query pattern
# P = "aga"
# wgi = WheelerGraphIndex(L, I, O, C, wt)
# low, high = wgi.backward_search(P)
# if low != -1:
#     matching_edges = wgi.paths_for_range(low, high)
#     print("Matching edges:", matching_edges)
# else:
#     print("No match found.")


In [ ]:
import sys

# Assuming your directory has these files:
# L (contains edge labels in order)
# I (bitvector for incoming edges, probably as a string of '0' and '1')
# O (bitvector for outgoing edges)
# nodes (mapping edges to nodes or vice versa)
# graph (some metadata or original graph information, optional)

def load_L(filename):
    # Assume L is stored as a single line of characters
    with open(filename, 'r') as f:
        L_str = f.read().strip()  # one string of characters
    return L_str

def load_bitvector(filename):
    # Assume I and O are stored as a string of '0'/'1' characters in a single line
    with open(filename, 'r') as f:
        bit_str = f.read().strip()
    return bit_str

def load_nodes(filename):
    # Assume nodes file might contain one node per line or a mapping of edges -> node
    # This will depend on your format. Let's assume a simple format: each line gives a node id for the corresponding edge index.
    with open(filename, 'r') as f:
        nodes = [line.strip() for line in f]
    return nodes


In [44]:
class RankSupport:
    """ A naive rank support example. For large data, implement something more efficient or use a wavelet tree. """
    def __init__(self, text):
        self.text = text
        # Precompute prefix counts for each character
        self.prefix_counts = {}
        # Collect all unique chars
        unique_chars = set(text)
        for c in unique_chars:
            self.prefix_counts[c] = [0] * (len(text) + 1)
        for i, ch in enumerate(text):
            for c in unique_chars:
                self.prefix_counts[c][i+1] = self.prefix_counts[c][i]
            self.prefix_counts[ch][i+1] += 1

    def rank(self, c, pos):
        # Returns number of occurrences of c in text[0..pos]
        # pos is inclusive index
        if c not in self.prefix_counts:
            return 0
        if pos < 0:
            return 0
        # prefix_counts[c][pos+1] because prefix_counts is 1-based indexing in this approach
        return self.prefix_counts[c][pos+1]



In [42]:
class WheelerGraphIndex:
    def __init__(self, L, I, O):
        self.L = L
        self.I = I
        self.O = O
        self.n = len(L)

        # Build a rank structure for L (wavelet tree or simple prefix rank)
        self.rank_support = RankSupport(L)

        # Build C array (like first in FM-index)
        # Count how many of each character
        char_counts = {}
        for ch in L:
            char_counts[ch] = char_counts.get(ch, 0) + 1

        # Calculate cumulative counts for C
        sorted_chars = sorted(char_counts.keys())
        self.C = {}
        total = 0
        for ch in sorted_chars:
            self.C[ch] = total
            total += char_counts[ch]

    def rank_c(self, c, pos):
        # rank of c in L up to position pos (0-based, inclusive)
        return self.rank_support.rank(c, pos)

    def backward_search(self, P):
        """Perform backward search for pattern P."""
        low, high = 0, self.n - 1
        for i in range(len(P) - 1, -1, -1):
            c = P[i]
            if c not in self.C:
                # Character doesn't appear at all
                return -1, -1

            lRank = self.rank_c(c, low - 1) if low > 0 else 0
            rRank = self.rank_c(c, high)

            if rRank == lRank:
                return -1, -1

            low = self.C[c] + lRank
            high = self.C[c] + rRank - 1

        return low, high

    def paths_for_range(self, low, high):
        # Here you would use I and O bitvectors (and nodes) to recover paths
        # For now, just return the edge indices
        return list(range(low, high+1))



In [43]:
def main():
    # Adjust filenames as needed
    L_file = "wheeler\out__rowboat\L.txt"       # contains the edge labels in order
    I_file = "wheeler\out__rowboat\I.txt"       # bitvector of incoming edges
    O_file = "wheeler\out__rowboat\O.txt"       # bitvector of outgoing edges
    nodes_file = r"wheeler\out__rowboat\nodes.txt"# node data (optional depending on use case)

    # Load data
    L_str = load_L(L_file)
    I_str = load_bitvector(I_file)
    O_str = load_bitvector(O_file)
    nodes = load_nodes(nodes_file)  # if needed

    # Construct the WheelerGraphIndex
    wgi = WheelerGraphIndex(L_str, I_str, O_str)

    # Example query
    P = "w"  # or any pattern you want to search
    low, high = wgi.backward_search(P)

    if low == -1:
        print(f"No occurrences of '{P}' found.")
    else:
        match_edges = wgi.paths_for_range(low, high)
        print(f"Matches of '{P}': edges {match_edges}")

if __name__ == "__main__":
    main()

Matches of 'w': edges [15, 16]
